In [ ]:
#可以实时用摄像头检测前方的东西（后续可以加开始和停止的时间+语音播报）

from ultralytics import YOLO
import cv2

# 加载 YOLOv8 模型
model = YOLO("yolov8n.pt")  # 确保 yolov8n.pt 在正确路径

# 初始化摄像头（0 表示默认摄像头，如果有多个摄像头可以尝试 1, 2, ...）
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ 摄像头无法打开")
    exit()

ret, frame = cap.read()
frame = cv2.resize(frame, (640, 480))  # 缩小分辨率提高速度

print("✅ 摄像头已开启，按 'q' 退出")

while True:
    # 读取摄像头画面
    ret, frame = cap.read()
    if not ret:
        print("❌ 无法读取摄像头画面")
        break

    # 使用 YOLOv8 进行目标检测
    results = model(frame, conf=0.4)  # conf 是置信度阈值

    # 获取图片的宽度和高度
    height, width = frame.shape[:2]

    # 遍历检测结果
    for result in results:
        for box in result.boxes:
            # 获取检测框坐标 (x1, y1, x2, y2)
            x1, y1, x2, y2 = box.xyxy[0].tolist()

            # 计算检测框的中心点
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2

            # 判断物体在画面中的位置（水平方向）
            if center_x < width / 3:
                x_position = "左前方"
            elif center_x > 2 * width / 3:
                x_position = "右前方"
            else:
                x_position = "正前方"

            # 判断物体距离（垂直方向）
            if center_y < height / 3:
                y_position = "近处"
            elif center_y > 2 * height / 3:
                y_position = "远处"
            else:
                y_position = "中距离"

            # 获取类别名称和置信度
            class_id = int(box.cls[0])
            class_name = result.names[class_id]
            confidence = float(box.conf[0])

            # 在画面上显示检测信息
            print(
                f"{class_name} ({confidence:.2f}) | {x_position}-{y_position}",
                (int(x1), int(y1) - 10),
            )

    # 显示检测结果
    #cv2.imshow("YOLOv8 实时检测 (按 q 退出)", frame)

    # 按 'q' 退出
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()